## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/85-googleq-a-train-glove-lstm/model.h5'
tokenizer_path = '/kaggle/input/85-googleq-a-train-glove-lstm/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 100
ANSWER_MAX_LEN = 100

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.3)(title)
title = Bidirectional(LSTM(128))(title)
title = Dropout(0.5)(title)
title = Dense(512)(title)
title = BatchNormalization()(title)
title_out = Activation('relu')(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.3)(body)
body = Bidirectional(LSTM(256, return_sequences=True))(body)
body = Bidirectional(LSTM(128))(body)
body = Dropout(0.5)(body)
body = Dense(512)(body)
body = BatchNormalization()(body)
body_out = Activation('relu')(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.3)(answer)
answer = Bidirectional(LSTM(256, return_sequences=True))(answer)
answer = Bidirectional(LSTM(128))(answer)
answer = Dropout(0.5)(answer)
answer = Dense(512)(answer)
answer = BatchNormalization()(answer)
answer_out = Activation('relu')(answer)

# Output
output = Concatenate()([title_out, body_out, answer_out])
output = Dense(512)(output)
output = BatchNormalization()(output)
output = Activation('relu')(output)
output = Dropout(0.5)(output)
output = Dense(N_CLASS, activation='sigmoid')(output)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_body_seq (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     17402100    input_body_seq[0][0]             
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.944269,0.676558,0.120995,0.720880,0.638757,0.695670,0.645332,0.588339,0.248523,...,0.919338,0.924362,0.636713,0.958738,0.970643,0.837432,0.072724,0.040552,0.712028,0.942164
1,46,0.920824,0.519511,0.000453,0.783923,0.818147,0.962993,0.546977,0.415150,0.023596,...,0.720977,0.974951,0.666343,0.990576,0.990907,0.912272,0.948294,0.070703,0.085700,0.933728
2,70,0.952608,0.761903,0.004602,0.851374,0.958950,0.967541,0.646351,0.574955,0.142392,...,0.930846,0.928126,0.617086,0.980159,0.979087,0.867342,0.038795,0.020746,0.782546,0.941020
3,132,0.930140,0.523690,0.000337,0.842065,0.862571,0.981397,0.536117,0.390071,0.039345,...,0.780562,0.980013,0.702481,0.992189,0.996708,0.936694,0.889507,0.067682,0.679347,0.957440
4,200,0.942827,0.441886,0.002978,0.767917,0.771335,0.886904,0.612098,0.500073,0.245955,...,0.782577,0.967325,0.662834,0.992021,0.988583,0.882758,0.530069,0.098648,0.596995,0.954017


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.915671,0.572820,0.018301,0.757304,0.839975,0.888508,0.583365,0.473683,0.161129,...,0.806329,0.959037,0.675792,0.982079,0.987578,0.897010,0.581743,0.096844,0.498370,0.937097
std,2812.670060,0.036351,0.140658,0.054119,0.118520,0.097609,0.132275,0.045003,0.077357,0.141000,...,0.089376,0.019165,0.033248,0.009912,0.008468,0.031283,0.331638,0.058413,0.278148,0.018421
min,39.000000,0.781842,0.299788,0.000109,0.332443,0.342479,0.234767,0.498546,0.345452,0.010092,...,0.553509,0.864802,0.576605,0.938275,0.934882,0.760113,0.003017,0.003726,0.030390,0.878445
25%,2572.000000,0.893647,0.447517,0.000837,0.690232,0.798045,0.887751,0.545918,0.415125,0.053696,...,0.733684,0.950097,0.652091,0.977971,0.984349,0.878550,0.247360,0.052150,0.237332,0.925818
50%,5093.000000,0.919265,0.565007,0.002039,0.771161,0.855479,0.939325,0.579272,0.452404,0.114563,...,0.810444,0.963892,0.677696,0.984522,0.989937,0.902440,0.697698,0.087434,0.481723,0.938525
75%,7482.000000,0.943719,0.686783,0.008858,0.849953,0.904422,0.961542,0.618381,0.527066,0.228235,...,0.889221,0.972688,0.700300,0.988957,0.993492,0.919987,0.880084,0.132597,0.723678,0.950202
max,9640.000000,0.982140,0.892080,0.553234,0.969186,0.995176,0.987549,0.724723,0.730084,0.723082,...,0.959908,0.989701,0.771454,0.996212,0.998511,0.963356,0.976569,0.299726,0.996244,0.974046
